In [1]:
import os
import pandas as pd
import numpy as np
import natsort
pd.set_option('display.max_rows', None)
from decimal import *
getcontext().prec = 2

from tkinter import *
import tkinter as tk
import tkinter.messagebox as messagebox
from functools import partial

In [2]:
import zipfile


def un_zip(file_name):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        zip_file.extract(names, file_name + "_files/")
    zip_file.close()

In [3]:
def processing(E1, E2, E3):
    try:
        # actual main()
        ali_files = E1.get()
        cle_file = E2.get()
        report_path = E3.get()

        cle_df = pre_process_cle(cle_file)
        raw_ali_df = merge_ali(ali_files)
        ali_df = pre_process_ali(raw_ali_df)

        report_df = CLE_to_ali(cle_df, ali_df)

        report_file_name = report_path + 'Report.csv'
        report_df.to_csv(report_file_name,
                         sep=',',
                         header=True,
                         index=True,
                         encoding='utf_8_sig')
        messagebox.showinfo('Congras', 'Done!')
    except ValueError:
        messagebox.showinfo('Warning', 'Wrong Input Path!')

In [3]:
def create_app():
    window = tk.Tk()

    L1 = Label(
        window,
        text="Customer Matcher version 0.1",
    ).grid(row=0, column=1)
    L2 = Label(
        window,
        text="Input path of Ali files",
    ).grid(row=1, column=0)
    L3 = Label(
        window,
        text="Input name of CLE",
    ).grid(row=2, column=0)
    L4 = Label(
        window,
        text="Input path of the report file",
    ).grid(row=3, column=0)

    E1 = Entry(window, bd=5)
    E1.grid(row=1, column=1)

    E2 = Entry(window, bd=5)
    E2.grid(row=2, column=1)

    E3 = Entry(window, bd=5)
    E3.grid(row=3, column=1)

    B = Button(window, text="Submit", command=partial(processing, E1, E2,
                                                      E3)).grid(
                                                          row=5,
                                                          column=1,
                                                      )
    window.mainloop()

In [4]:
'''
    path is for file dic
'''
def un_zip_all(path):
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            file_name = os.path.join(root, name)
            if (file_name.endswith('.zip')):
                print(file_name)
#                 un_zip(file_name)

# print(un_zip_all('../Desktop/fossil2020'))

In [5]:
'''
    Ali files are scv files.
    return a dataframe.
'''
def read_ali(file_name):
    file = pd.read_csv(open(file_name),skiprows=4,header=0,index_col=0,
        names=['账务流水号','业务流水号','商户订单号','商品名称','发生时间','对方账号','收入金额',
      '支出金额（-元）','账户余额（元）','交易渠道',
      '业务类型','备注','业务描述','业务账单来源','业务基础订单号','业务订单号'])
    # drop all data when ID is nan

    file = file.dropna(subset = ['业务基础订单号'])
    df = pd.DataFrame(file)
    df['业务基础订单号'] = df['业务基础订单号'].str.strip('\t')
    df_short = df[['业务基础订单号','收入金额','支出金额（-元）','业务类型']]
    return df_short

In [6]:
'''
    CLE files are xlsx files.
    return a dataframe.
'''
def read_CLE(file_name):
    file = pd.read_excel(file_name)
    file = file.dropna(subset = ['External Document No.'])
    df = pd.DataFrame(file)
    df_short = df[['External Document No.','Posting Date','Document Type','Customer No.','Amount','Open','User ID']]
    print(file_name + 'Done!')
    return df_short

In [7]:
'''
    Before use CLE data, need to count overall price for each ID.
    input file and use numpy array;
    sort by IDs; use loop to check and count; delete other price, only remain
    the main ID.
'''


def pre_process_cle(file_name):
    file = pd.read_excel(file_name)
    file = file.dropna(subset=['External Document No.'])
    df = pd.DataFrame(file)
    df = df[[
        'External Document No.', 'Posting Date', 'Document Type',
        'Customer No.', 'Amount', 'Open', 'User ID'
    ]]

    df_duplicated = df[df['External Document No.'].duplicated()]
    print("Found duplicated error.")
    print(df_duplicated)
    df = df[~df['External Document No.'].duplicated()]
    df = df.sort_values(by='External Document No.')

    array = np.array(df)
    new_cle_df = pd.DataFrame(columns=[
        'External Doc No.',
        'Posting Date',
        'Doc Type',
        'Customer No.',
        'Amount',
        'Open',
        'User ID',
    ])
    i = 0
    while i < len(array):
        # Now only check official SUDO127, so pass other customer
        if (array[i, 3] != 'SUD0127'):
            i = i + 1
            continue
        orderid = str(array[i, 0])
        if True:
            # this is a header
            head_pos = i
            count = []
            count.append(array[head_pos, 4])

            for follower_pos in range(head_pos + 1, len(array)):
                follower_id = str(array[follower_pos, 0])
                # print('follower_id is: '+str(follower_id))
                if orderid in follower_id:
                    count.append(array[follower_pos, 4])
                else:
                    break

            # count overall value
            result_amount = 0.0

            for v in range(0, len(count)):
                # print(str(v)+' : '+str(count[v]))
                result_amount = result_amount + count[v]
            # save 2 .00
            result_amount = round(result_amount, 2)
            # print('now result value is : '+str(result_amount))
            # save new data
            new_data = {
                'External Doc No.': str(orderid),
                'Posting Date': array[i, 1],
                'Doc Type': array[i, 2],
                'Customer No.': array[i, 3],
                'Amount': result_amount,
                'Open': array[i, 5],
                'User ID': array[i, 6]
            }
            new_cle_df = new_cle_df.append(new_data, ignore_index=True)

            # update i
            #             print('now len of count is : '+str(len(count)))
            i = i + len(count)


#     new_cle_df.to_excel(new_cle_file_path, header=True, index=True,encoding='utf_8_sig')
    print('Processing ' + file_name + ' done!')
    return new_cle_df

In [8]:
'''
    After call function merge_ali,
    need to confine ali_df;
    For single ali_id, When ali type are '交易付款' or '在线支付' or '交易退款'
    Count these three values into one result_value.
    return a dataframe.
'''
def pre_process_ali(ali_df):
    ali_array = np.array(ali_df)
    ali_df_merge = pd.DataFrame(columns=['业务基础订单号','收入金额','支出金额（-元）','业务类型'])
    
    while(len(ali_array!=0)):
        ali_ID = ali_array[0,0]
        # check whether this id has already in ali_df_merge
        temp_ali = np.array(ali_df_merge)
        check = np.argwhere(temp_ali==ali_ID)
        
        if(len(check)==0):
            # print('Now dealing with '+ali_ID)
            locs = np.argwhere(ali_array==ali_ID)
            count = 0.0
            del_index = []
            for loc in locs:
                index = loc[0]
                del_index.append(index)
                ali_income = ali_array[index,1]
                ali_outcome = ali_array[index,2]
                ali_type = ali_array[index,3]
                if(ali_type=='交易付款' or ali_type=='在线支付' or ali_type == '交易退款'):
                    # this is a value needed to be count
                    count = count + ali_income + ali_outcome
                    
                else:
                    continue
            count = round(count,2)
            
            # save this ID and new counted price
            if count >= 0:
                new_data = {
                        '业务基础订单号':ali_ID,
                        '收入金额' : count,
                        '支出金额（-元）' : 0.0,
                        '业务类型' : 'Counted Price'
                }
                ali_df_merge = ali_df_merge.append(new_data,ignore_index=True)
            else:
                new_data = {
                        '业务基础订单号':ali_ID,
                        '收入金额' : 0.0,
                        '支出金额（-元）' : count,
                        '业务类型' : 'Counted Price'
                }
                ali_df_merge = ali_df_merge.append(new_data,ignore_index=True)
            
            # delete related index
#             print(ali_array[0])
            ali_array = np.delete(ali_array, del_index, axis=0)
            

               
        else:
            # this id has been counted before, no need to count, just pass it.
            continue
        # print(ali_ID+" ended.")
    print('Prepocess Ali Done!')
    return ali_df_merge
       

In [9]:
'''
    Merge all Ali data into one Dataframe;
    path
    return a dataframe.
'''
def merge_ali(path):
    result_df = pd.DataFrame(columns=['业务基础订单号','收入金额','支出金额（-元）','业务类型'])
    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            file_name = os.path.join(root, name)
            if (file_name.endswith('.csv') and ('╒╦╬±├≈╧╕' in file_name ) and 
               ('╒╦╬±├≈╧╕(╗π╫▄)' not in file_name)):
                print('Now dealing with '+file_name)
                temp_df = read_ali(file_name)
                result_df = pd.concat([result_df,temp_df],ignore_index = True)
    result_df.set_index(["业务基础订单号"],)
    
    return result_df

In [10]:
'''
    For efficiency consideration, use numpy to process data;
    Turn dataframe into 2-dimensional numpy;
    use the ID in CLE, find the ID in ali, result is a two dimensional list;
    [index][0] , use this to get income and outcome and ready to compare;
    Two bps: 1,no ID in ali; 2, income or outcome cannot match.
    return a dataframe to report.
'''
def CLE_to_ali(cle_df,ali_df):
    cle_array = np.array(cle_df)
    ali_array = np.array(ali_df)
    report_df = pd.DataFrame(columns=['Reason','External Doc No.','Posting Date','Doc Type',
        'Customer No.','Amount','UserID','Open','Ali_Amount','Ali_Type'])
    
    print('Start checking CLE to Ali data.')
    # loop cle_array and search ali_array
    for i in range(0,len(cle_array)):
        print('Now check CLE num'+str(i))
        ID = cle_array[i,0]
        
        # find index location by ID, this loc suppose to be unique
        locs = np.argwhere(ali_array==ID)
        
        
        # if doesn't exist
        if len(locs)==0:
            new_data = {
                'Reason':'Empty',
                'External Doc No.' : str(cle_array[i,0])+'\t',
                'Posting Date' : cle_array[i,1],
                'Doc Type' : cle_array[i,2],
                'Customer No.' : cle_array[i,3],
                'Amount' : cle_array[i,4],
                'Open' : cle_array[i,5]
            }
            report_df = report_df.append(new_data,ignore_index=True)
        else:
            # multiple transaction in one case, need to add 3 types into all;
            report_amount = 0.0
            for loc in locs:
                    
                index = loc[0]
                
                ali_ID = ali_array[index,0]
                if ali_array[index,1]==0 and ali_array[index,2]==0:
                    report_amount = report_amount
                elif ali_array[index,1]==0 and ali_array[index,2]!=0:
                    report_amount = report_amount + ali_array[index,2]
                else:
                    report_amount = report_amount + ali_array[index,1]
                        
            report_amount = round(report_amount, 2)
            # only report priceError cases;
            if(report_amount != cle_array[i,4]):

                new_data = {
                            'Reason':'PriceError',
                            'External Doc No.' : str(cle_array[i,0])+'\t',
                            'Posting Date' : cle_array[i,1],
                            'Doc Type' : cle_array[i,2],
                            'Customer No.' : cle_array[i,3],
                            'Amount' : cle_array[i,4],
                            'Open' : cle_array[i,5],
                            'Ali_Amount' : report_amount,
                            'Ali_Type' : ali_array[index,3]
                }
                report_df = report_df.append(new_data,ignore_index=True)
    return report_df

In [23]:
<div class="burk">
'''
    For efficiency consideration, use numpy to process data;
    Turn dataframe into 2-dimensional numpy;
    
'''
def ali_to_CLE(ali_df,cle_df):
    cle_array = np.array(cle_df)
    ali_array = np.array(ali_df)
    report_df = pd.DataFrame(columns=['Reason','External Doc No.','Posting Date','Doc Type',
        'Customer No.','Amount','UserID','Open','Ali_Amount','Ali_Type'])
    
    print('Start checking Ali data to Ali CLE.')
    for i in range(0,len(ali_array)):
        print('Now check ALI ID : '+str(i))
        ID = ali_array[i,0]
        locs = np.argwhere(cle_array==ID)
        
        # ali price have been counted already, label is Counted Price
        # len of locs should be 1, because CLE has been pre-processed
        if(len(locs)>1):
            print('Found an error, there are multiple same External Doc No., pre-process-CLE has problems.')
            break
            
        report_amount = 0.0
        if(ali_array[i,1]>0):
            report_amount = ali_array[i,1]
        else:
            report_amount = ali_array[i,2]
        # if doesn't exist in CLE

        if len(locs)==0:
            if(report_amount!=0):
                # CLE only contains non-0 price, so we discard IDs when income and outcome both equal 0;
                new_data = {
                'Reason':'Empty',
                'External Doc No.' : ID+'\t',
                'Ali_Amount' : report_amount,
                'Ali_Type' : ali_array[i,3]
                }
                report_df = report_df.append(new_data,ignore_index=True)
        else:
        # exist in CLE
            index = locs[0][0]
            if(report_amount != cle_array[index,4]):
                new_data = {
                            'Reason':'PriceError',
                            'External Doc No.' : str(cle_array[index,0])+'\t',
                            'Posting Date' : cle_array[index,1],
                            'Doc Type' : cle_array[index,2],
                            'Customer No.' : cle_array[index,3],
                            'Amount' : cle_array[index,4],
                            'Open' : cle_array[index,5],
                            'Ali_Amount' : report_amount,
                            'Ali_Type' : ali_array[i,3]
                }
                report_df = report_df.append(new_data,ignore_index=True)
    report_df.sort('Posting Date')
    return report_df</div><i class="fa fa-lightbulb-o "></i>

In [24]:
'''
    check for single ID in ali pay;
'''

ali_array = np.array(raw_ali_df)
print(ali_array)
locs = np.argwhere(ali_array == '1133682721279123307')
print(locs)
for loc in locs:
    index = loc[0]
    print(ali_array[index, 0] + ',' + str(ali_array[index, 1]) + ',' +
          str(ali_array[index, 2]) + ',' + ali_array[index, 3])

[['1133682721279123307' 590.81 0.0 '交易付款']
 ['1133682721279123307' 0.0 -4.73 '收费']
 ['1133682721279123307' 0.0 -2.95 '转账']
 ...
 ['1458702758697235218' 0.0 -0.02 '交易分账']
 ['1458702758697235218' 0.0 -30.65 '交易分账']
 ['1239147914967949287' 1.0 0.0 '交易付款']]
[[0 0]
 [1 0]
 [2 0]
 [3 0]
 [4 0]
 [5 0]
 [6 0]]
1133682721279123307,590.81,0.0,交易付款
1133682721279123307,0.0,-4.73,收费
1133682721279123307,0.0,-2.95,转账
1133682721279123307,0.0,-29.54,转账
1133682721279123307,0.0,-59.08,交易分账
1133682721279123307,0.0,-0.02,交易分账
1133682721279123307,0.0,-29.54,交易分账


In [12]:
def main():
    cle_df = pre_process_cle('../Desktop/fossil2020/Customer Ledger Entries FOSSIL_LINDA.xlsx','./test/CLE1.xlsx')
    raw_ali_df = merge_ali('./test/8-12/')
    ali_df = pre_process_ali(raw_ali_df)

    report_df = CLE_to_ali(cle_df,ali_df)
    report_df.to_csv('./test/8-12/report2.csv', sep=',', header=True, index=True,encoding='utf_8_sig')
if __name__ == '__main__':
    try:
        main()
        print('Done!')
    except Exception as e:
        print('Error: {}'.format(e))

Found duplicated error.
     External Document No. Posting Date Document Type Customer No.  Amount  \
3602   1160123617127096229   2020-08-14       Invoice      SUD0127  2277.0   

       Open           User ID  
3602  False  FOSSIL\NASSERVER  
Processing ../Desktop/fossil2020/Customer Ledger Entries FOSSIL_LINDA.xlsx done!
Now dealing with ./test/8-12/202008_2088311966171753.zip_files\20883119661717530156_202008_╒╦╬±├≈╧╕_1.csv
Now dealing with ./test/8-12/202009_2088311966171753.zip_files\20883119661717530156_202009_╒╦╬±├≈╧╕_1.csv
Now dealing with ./test/8-12/202010_2088311966171753.zip_files\20883119661717530156_202010_╒╦╬±├≈╧╕_1.csv
Now dealing with ./test/8-12/202011_2088311966171753.zip_files\20883119661717530156_202011_╒╦╬±├≈╧╕_1.csv
Now dealing with ./test/8-12/202012_2088311966171753.zip_files\20883119661717530156_202012_╒╦╬±├≈╧╕_1.csv
Prepocess Ali Done!
Start checking CLE to Ali data.
Now check CLE num0
Now check CLE num1
Now check CLE num2
Now check CLE num3
Now check CLE nu

Now check CLE num387
Now check CLE num388
Now check CLE num389
Now check CLE num390
Now check CLE num391
Now check CLE num392
Now check CLE num393
Now check CLE num394
Now check CLE num395
Now check CLE num396
Now check CLE num397
Now check CLE num398
Now check CLE num399
Now check CLE num400
Now check CLE num401
Now check CLE num402
Now check CLE num403
Now check CLE num404
Now check CLE num405
Now check CLE num406
Now check CLE num407
Now check CLE num408
Now check CLE num409
Now check CLE num410
Now check CLE num411
Now check CLE num412
Now check CLE num413
Now check CLE num414
Now check CLE num415
Now check CLE num416
Now check CLE num417
Now check CLE num418
Now check CLE num419
Now check CLE num420
Now check CLE num421
Now check CLE num422
Now check CLE num423
Now check CLE num424
Now check CLE num425
Now check CLE num426
Now check CLE num427
Now check CLE num428
Now check CLE num429
Now check CLE num430
Now check CLE num431
Now check CLE num432
Now check CLE num433
Now check CLE

Now check CLE num822
Now check CLE num823
Now check CLE num824
Now check CLE num825
Now check CLE num826
Now check CLE num827
Now check CLE num828
Now check CLE num829
Now check CLE num830
Now check CLE num831
Now check CLE num832
Now check CLE num833
Now check CLE num834
Now check CLE num835
Now check CLE num836
Now check CLE num837
Now check CLE num838
Now check CLE num839
Now check CLE num840
Now check CLE num841
Now check CLE num842
Now check CLE num843
Now check CLE num844
Now check CLE num845
Now check CLE num846
Now check CLE num847
Now check CLE num848
Now check CLE num849
Now check CLE num850
Now check CLE num851
Now check CLE num852
Now check CLE num853
Now check CLE num854
Now check CLE num855
Now check CLE num856
Now check CLE num857
Now check CLE num858
Now check CLE num859
Now check CLE num860
Now check CLE num861
Now check CLE num862
Now check CLE num863
Now check CLE num864
Now check CLE num865
Now check CLE num866
Now check CLE num867
Now check CLE num868
Now check CLE

Now check CLE num1247
Now check CLE num1248
Now check CLE num1249
Now check CLE num1250
Now check CLE num1251
Now check CLE num1252
Now check CLE num1253
Now check CLE num1254
Now check CLE num1255
Now check CLE num1256
Now check CLE num1257
Now check CLE num1258
Now check CLE num1259
Now check CLE num1260
Now check CLE num1261
Now check CLE num1262
Now check CLE num1263
Now check CLE num1264
Now check CLE num1265
Now check CLE num1266
Now check CLE num1267
Now check CLE num1268
Now check CLE num1269
Now check CLE num1270
Now check CLE num1271
Now check CLE num1272
Now check CLE num1273
Now check CLE num1274
Now check CLE num1275
Now check CLE num1276
Now check CLE num1277
Now check CLE num1278
Now check CLE num1279
Now check CLE num1280
Now check CLE num1281
Now check CLE num1282
Now check CLE num1283
Now check CLE num1284
Now check CLE num1285
Now check CLE num1286
Now check CLE num1287
Now check CLE num1288
Now check CLE num1289
Now check CLE num1290
Now check CLE num1291
Now check 

Now check CLE num1620
Now check CLE num1621
Now check CLE num1622
Now check CLE num1623
Now check CLE num1624
Now check CLE num1625
Now check CLE num1626
Now check CLE num1627
Now check CLE num1628
Now check CLE num1629
Now check CLE num1630
Now check CLE num1631
Now check CLE num1632
Now check CLE num1633
Now check CLE num1634
Now check CLE num1635
Now check CLE num1636
Now check CLE num1637
Now check CLE num1638
Now check CLE num1639
Now check CLE num1640
Now check CLE num1641
Now check CLE num1642
Now check CLE num1643
Now check CLE num1644
Now check CLE num1645
Now check CLE num1646
Now check CLE num1647
Now check CLE num1648
Now check CLE num1649
Now check CLE num1650
Now check CLE num1651
Now check CLE num1652
Now check CLE num1653
Now check CLE num1654
Now check CLE num1655
Now check CLE num1656
Now check CLE num1657
Now check CLE num1658
Now check CLE num1659
Now check CLE num1660
Now check CLE num1661
Now check CLE num1662
Now check CLE num1663
Now check CLE num1664
Now check 

...

Now check CLE num20317
Now check CLE num20318
Now check CLE num20319
Now check CLE num20320
Now check CLE num20321
Now check CLE num20322
Now check CLE num20323
Now check CLE num20324
Now check CLE num20325
Now check CLE num20326
Now check CLE num20327
Now check CLE num20328
Now check CLE num20329
Now check CLE num20330
Now check CLE num20331
Now check CLE num20332
Now check CLE num20333
Now check CLE num20334
Now check CLE num20335
Now check CLE num20336
Now check CLE num20337
Now check CLE num20338
Now check CLE num20339
Now check CLE num20340
Now check CLE num20341
Now check CLE num20342
Now check CLE num20343
Now check CLE num20344
Now check CLE num20345
Now check CLE num20346
Now check CLE num20347
Now check CLE num20348
Now check CLE num20349
Now check CLE num20350
Now check CLE num20351
Now check CLE num20352
Now check CLE num20353
Now check CLE num20354
Now check CLE num20355
Now check CLE num20356
Now check CLE num20357
Now check CLE num20358
Now check CLE num20359
Now check C

Now check CLE num20689
Now check CLE num20690
Now check CLE num20691
Now check CLE num20692
Now check CLE num20693
Now check CLE num20694
Now check CLE num20695
Now check CLE num20696
Now check CLE num20697
Now check CLE num20698
Now check CLE num20699
Now check CLE num20700
Now check CLE num20701
Now check CLE num20702
Now check CLE num20703
Now check CLE num20704
Now check CLE num20705
Now check CLE num20706
Now check CLE num20707
Now check CLE num20708
Now check CLE num20709
Now check CLE num20710
Now check CLE num20711
Now check CLE num20712
Now check CLE num20713
Now check CLE num20714
Now check CLE num20715
Now check CLE num20716
Now check CLE num20717
Now check CLE num20718
Now check CLE num20719
Now check CLE num20720
Now check CLE num20721
Now check CLE num20722
Now check CLE num20723
Now check CLE num20724
Now check CLE num20725
Now check CLE num20726
Now check CLE num20727
Now check CLE num20728
Now check CLE num20729
Now check CLE num20730
Now check CLE num20731
Now check C

Now check CLE num21060
Now check CLE num21061
Now check CLE num21062
Now check CLE num21063
Now check CLE num21064
Now check CLE num21065
Now check CLE num21066
Now check CLE num21067
Now check CLE num21068
Now check CLE num21069
Now check CLE num21070
Now check CLE num21071
Now check CLE num21072
Now check CLE num21073
Now check CLE num21074
Now check CLE num21075
Now check CLE num21076
Now check CLE num21077
Now check CLE num21078
Now check CLE num21079
Now check CLE num21080
Now check CLE num21081
Now check CLE num21082
Now check CLE num21083
Now check CLE num21084
Now check CLE num21085
Now check CLE num21086
Now check CLE num21087
Now check CLE num21088
Now check CLE num21089
Now check CLE num21090
Now check CLE num21091
Now check CLE num21092
Now check CLE num21093
Now check CLE num21094
Now check CLE num21095
Now check CLE num21096
Now check CLE num21097
Now check CLE num21098
Now check CLE num21099
Now check CLE num21100
Now check CLE num21101
Now check CLE num21102
Now check C

Now check CLE num21420
Now check CLE num21421
Now check CLE num21422
Now check CLE num21423
Now check CLE num21424
Now check CLE num21425
Now check CLE num21426
Now check CLE num21427
Now check CLE num21428
Now check CLE num21429
Now check CLE num21430
Now check CLE num21431
Now check CLE num21432
Now check CLE num21433
Now check CLE num21434
Now check CLE num21435
Now check CLE num21436
Now check CLE num21437
Now check CLE num21438
Now check CLE num21439
Now check CLE num21440
Now check CLE num21441
Now check CLE num21442
Now check CLE num21443
Now check CLE num21444
Now check CLE num21445
Now check CLE num21446
Now check CLE num21447
Now check CLE num21448
Now check CLE num21449
Now check CLE num21450
Now check CLE num21451
Now check CLE num21452
Now check CLE num21453
Now check CLE num21454
Now check CLE num21455
Now check CLE num21456
Now check CLE num21457
Now check CLE num21458
Now check CLE num21459
Now check CLE num21460
Now check CLE num21461
Now check CLE num21462
Now check C

Done!


In [14]:
# check pre_process_cle is right.
cle_df = pre_process_cle('../../../Desktop/fossil2020/Customer Ledger Entries FOSSIL_LINDA.xlsx')


Found duplicated error.
     External Document No. Posting Date Document Type Customer No.  Amount  \
3602   1160123617127096229   2020-08-14       Invoice      SUD0127  2277.0   

       Open           User ID  
3602  False  FOSSIL\NASSERVER  
Processing ../../../Desktop/fossil2020/Customer Ledger Entries FOSSIL_LINDA.xlsx done!


In [15]:
raw_ali_df = merge_ali('../../../Desktop/test/8-12/')
ali_df = pre_process_ali(raw_ali_df)

Now dealing with ../../../Desktop/test/8-12/202008_2088311966171753.zip_files\20883119661717530156_202008_╒╦╬±├≈╧╕_1.csv
Now dealing with ../../../Desktop/test/8-12/202009_2088311966171753.zip_files\20883119661717530156_202009_╒╦╬±├≈╧╕_1.csv
Now dealing with ../../../Desktop/test/8-12/202010_2088311966171753.zip_files\20883119661717530156_202010_╒╦╬±├≈╧╕_1.csv
Now dealing with ../../../Desktop/test/8-12/202011_2088311966171753.zip_files\20883119661717530156_202011_╒╦╬±├≈╧╕_1.csv
Now dealing with ../../../Desktop/test/8-12/202012_2088311966171753.zip_files\20883119661717530156_202012_╒╦╬±├≈╧╕_1.csv
Prepocess Ali Done!


In [26]:
report_df = CLE_to_ali(cle_df,ali_df)
report_df.to_csv('../../../Desktop/test/8-12/report4.csv', sep=',', header=True, index=True,encoding='utf_8_sig')

Start checking CLE to Ali data.
Now check CLE num0
Now check CLE num1
Now check CLE num2
Now check CLE num3
Now check CLE num4
Now check CLE num5
Now check CLE num6
Now check CLE num7
Now check CLE num8
Now check CLE num9
Now check CLE num10
Now check CLE num11
Now check CLE num12
Now check CLE num13
Now check CLE num14
Now check CLE num15
Now check CLE num16
Now check CLE num17
Now check CLE num18
Now check CLE num19
Now check CLE num20
Now check CLE num21
Now check CLE num22
Now check CLE num23
Now check CLE num24
Now check CLE num25
Now check CLE num26
Now check CLE num27
Now check CLE num28
Now check CLE num29
Now check CLE num30
Now check CLE num31
Now check CLE num32
Now check CLE num33
Now check CLE num34
Now check CLE num35
Now check CLE num36
Now check CLE num37
Now check CLE num38
Now check CLE num39
Now check CLE num40
Now check CLE num41
Now check CLE num42
Now check CLE num43
Now check CLE num44
Now check CLE num45
Now check CLE num46
Now check CLE num47
Now check CLE num4

Now check CLE num431
Now check CLE num432
Now check CLE num433
Now check CLE num434
Now check CLE num435
Now check CLE num436
Now check CLE num437
Now check CLE num438
Now check CLE num439
Now check CLE num440
Now check CLE num441
Now check CLE num442
Now check CLE num443
Now check CLE num444
Now check CLE num445
Now check CLE num446
Now check CLE num447
Now check CLE num448
Now check CLE num449
Now check CLE num450
Now check CLE num451
Now check CLE num452
Now check CLE num453
Now check CLE num454
Now check CLE num455
Now check CLE num456
Now check CLE num457
Now check CLE num458
Now check CLE num459
Now check CLE num460
Now check CLE num461
Now check CLE num462
Now check CLE num463
Now check CLE num464
Now check CLE num465
Now check CLE num466
Now check CLE num467
Now check CLE num468
Now check CLE num469
Now check CLE num470
Now check CLE num471
Now check CLE num472
Now check CLE num473
Now check CLE num474
Now check CLE num475
Now check CLE num476
Now check CLE num477
Now check CLE

Now check CLE num845
Now check CLE num846
Now check CLE num847
Now check CLE num848
Now check CLE num849
Now check CLE num850
Now check CLE num851
Now check CLE num852
Now check CLE num853
Now check CLE num854
Now check CLE num855
Now check CLE num856
Now check CLE num857
Now check CLE num858
Now check CLE num859
Now check CLE num860
Now check CLE num861
Now check CLE num862
Now check CLE num863
Now check CLE num864
Now check CLE num865
Now check CLE num866
Now check CLE num867
Now check CLE num868
Now check CLE num869
Now check CLE num870
Now check CLE num871
Now check CLE num872
Now check CLE num873
Now check CLE num874
Now check CLE num875
Now check CLE num876
Now check CLE num877
Now check CLE num878
Now check CLE num879
Now check CLE num880
Now check CLE num881
Now check CLE num882
Now check CLE num883
Now check CLE num884
Now check CLE num885
Now check CLE num886
Now check CLE num887
Now check CLE num888
Now check CLE num889
Now check CLE num890
Now check CLE num891
Now check CLE

Now check CLE num1287
Now check CLE num1288
Now check CLE num1289
Now check CLE num1290
Now check CLE num1291
Now check CLE num1292
Now check CLE num1293
Now check CLE num1294
Now check CLE num1295
Now check CLE num1296
Now check CLE num1297
Now check CLE num1298
Now check CLE num1299
Now check CLE num1300
Now check CLE num1301
Now check CLE num1302
Now check CLE num1303
Now check CLE num1304
Now check CLE num1305
Now check CLE num1306
Now check CLE num1307
Now check CLE num1308
Now check CLE num1309
Now check CLE num1310
Now check CLE num1311
Now check CLE num1312
Now check CLE num1313
Now check CLE num1314
Now check CLE num1315
Now check CLE num1316
Now check CLE num1317
Now check CLE num1318
Now check CLE num1319
Now check CLE num1320
Now check CLE num1321
Now check CLE num1322
Now check CLE num1323
Now check CLE num1324
Now check CLE num1325
Now check CLE num1326
Now check CLE num1327
Now check CLE num1328
Now check CLE num1329
Now check CLE num1330
Now check CLE num1331
Now check 

Now check CLE num1727
Now check CLE num1728
Now check CLE num1729
Now check CLE num1730
Now check CLE num1731
Now check CLE num1732
Now check CLE num1733
Now check CLE num1734
Now check CLE num1735
Now check CLE num1736
Now check CLE num1737
Now check CLE num1738
Now check CLE num1739
Now check CLE num1740
Now check CLE num1741
Now check CLE num1742
Now check CLE num1743
Now check CLE num1744
Now check CLE num1745
Now check CLE num1746
Now check CLE num1747
Now check CLE num1748
Now check CLE num1749
Now check CLE num1750
Now check CLE num1751
Now check CLE num1752
Now check CLE num1753
Now check CLE num1754
Now check CLE num1755
Now check CLE num1756
Now check CLE num1757
Now check CLE num1758
Now check CLE num1759
Now check CLE num1760
Now check CLE num1761
Now check CLE num1762
Now check CLE num1763
Now check CLE num1764
Now check CLE num1765
Now check CLE num1766
Now check CLE num1767
Now check CLE num1768
Now check CLE num1769
Now check CLE num1770
Now check CLE num1771
Now check 

...

Now check CLE num7972
Now check CLE num7973
Now check CLE num7974
Now check CLE num7975
Now check CLE num7976
Now check CLE num7977
Now check CLE num7978
Now check CLE num7979
Now check CLE num7980
Now check CLE num7981
Now check CLE num7982
Now check CLE num7983
Now check CLE num7984
Now check CLE num7985
Now check CLE num7986
Now check CLE num7987
Now check CLE num7988
Now check CLE num7989
Now check CLE num7990
Now check CLE num7991
Now check CLE num7992
Now check CLE num7993
Now check CLE num7994
Now check CLE num7995
Now check CLE num7996
Now check CLE num7997
Now check CLE num7998
Now check CLE num7999
Now check CLE num8000
Now check CLE num8001
Now check CLE num8002
Now check CLE num8003
Now check CLE num8004
Now check CLE num8005
Now check CLE num8006
Now check CLE num8007
Now check CLE num8008
Now check CLE num8009
Now check CLE num8010
Now check CLE num8011
Now check CLE num8012
Now check CLE num8013
Now check CLE num8014
Now check CLE num8015
Now check CLE num8016
Now check 

Now check CLE num8355
Now check CLE num8356
Now check CLE num8357
Now check CLE num8358
Now check CLE num8359
Now check CLE num8360
Now check CLE num8361
Now check CLE num8362
Now check CLE num8363
Now check CLE num8364
Now check CLE num8365
Now check CLE num8366
Now check CLE num8367
Now check CLE num8368
Now check CLE num8369
Now check CLE num8370
Now check CLE num8371
Now check CLE num8372
Now check CLE num8373
Now check CLE num8374
Now check CLE num8375
Now check CLE num8376
Now check CLE num8377
Now check CLE num8378
Now check CLE num8379
Now check CLE num8380
Now check CLE num8381
Now check CLE num8382
Now check CLE num8383
Now check CLE num8384
Now check CLE num8385
Now check CLE num8386
Now check CLE num8387
Now check CLE num8388
Now check CLE num8389
Now check CLE num8390
Now check CLE num8391
Now check CLE num8392
Now check CLE num8393
Now check CLE num8394
Now check CLE num8395
Now check CLE num8396
Now check CLE num8397
Now check CLE num8398
Now check CLE num8399
Now check 

Now check CLE num8771
Now check CLE num8772
Now check CLE num8773
Now check CLE num8774
Now check CLE num8775
Now check CLE num8776
Now check CLE num8777
Now check CLE num8778
Now check CLE num8779
Now check CLE num8780
Now check CLE num8781
Now check CLE num8782
Now check CLE num8783
Now check CLE num8784
Now check CLE num8785
Now check CLE num8786
Now check CLE num8787
Now check CLE num8788
Now check CLE num8789
Now check CLE num8790
Now check CLE num8791
Now check CLE num8792
Now check CLE num8793
Now check CLE num8794
Now check CLE num8795
Now check CLE num8796
Now check CLE num8797
Now check CLE num8798
Now check CLE num8799
Now check CLE num8800
Now check CLE num8801
Now check CLE num8802
Now check CLE num8803
Now check CLE num8804
Now check CLE num8805
Now check CLE num8806
Now check CLE num8807
Now check CLE num8808
Now check CLE num8809
Now check CLE num8810
Now check CLE num8811
Now check CLE num8812
Now check CLE num8813
Now check CLE num8814
Now check CLE num8815
Now check 

Now check CLE num9145
Now check CLE num9146
Now check CLE num9147
Now check CLE num9148
Now check CLE num9149
Now check CLE num9150
Now check CLE num9151
Now check CLE num9152
Now check CLE num9153
Now check CLE num9154
Now check CLE num9155
Now check CLE num9156
Now check CLE num9157
Now check CLE num9158
Now check CLE num9159
Now check CLE num9160
Now check CLE num9161
Now check CLE num9162
Now check CLE num9163
Now check CLE num9164
Now check CLE num9165
Now check CLE num9166
Now check CLE num9167
Now check CLE num9168
Now check CLE num9169
Now check CLE num9170
Now check CLE num9171
Now check CLE num9172
Now check CLE num9173
Now check CLE num9174
Now check CLE num9175
Now check CLE num9176
Now check CLE num9177
Now check CLE num9178
Now check CLE num9179
Now check CLE num9180
Now check CLE num9181
Now check CLE num9182
Now check CLE num9183
Now check CLE num9184
Now check CLE num9185
Now check CLE num9186
Now check CLE num9187
Now check CLE num9188
Now check CLE num9189
Now check 

Now check CLE num9527
Now check CLE num9528
Now check CLE num9529
Now check CLE num9530
Now check CLE num9531
Now check CLE num9532
Now check CLE num9533
Now check CLE num9534
Now check CLE num9535
Now check CLE num9536
Now check CLE num9537
Now check CLE num9538
Now check CLE num9539
Now check CLE num9540
Now check CLE num9541
Now check CLE num9542
Now check CLE num9543
Now check CLE num9544
Now check CLE num9545
Now check CLE num9546
Now check CLE num9547
Now check CLE num9548
Now check CLE num9549
Now check CLE num9550
Now check CLE num9551
Now check CLE num9552
Now check CLE num9553
Now check CLE num9554
Now check CLE num9555
Now check CLE num9556
Now check CLE num9557
Now check CLE num9558
Now check CLE num9559
Now check CLE num9560
Now check CLE num9561
Now check CLE num9562
Now check CLE num9563
Now check CLE num9564
Now check CLE num9565
Now check CLE num9566
Now check CLE num9567
Now check CLE num9568
Now check CLE num9569
Now check CLE num9570
Now check CLE num9571
Now check 

In [9]:
create_app()

In [ ]:
'''
../../../Desktop/test/8-12/
../../../Desktop/fossil2020/Customer Ledger Entries FOSSIL_LINDA.xlsx
../../../Desktop/Processor/
'''

In [22]:
report_df = ali_to_CLE(ali_df,cle_df)
report_df.to_csv('../../../Desktop/test/8-12/report_ali_to_cle3.csv', sep=',', header=True, index=True,encoding='utf_8_sig')

Start checking Ali data to Ali CLE.
Now check ALI ID : 0
Now check ALI ID : 1
Now check ALI ID : 2
Now check ALI ID : 3
Now check ALI ID : 4
Now check ALI ID : 5
Now check ALI ID : 6
Now check ALI ID : 7
Now check ALI ID : 8
Now check ALI ID : 9
Now check ALI ID : 10
Now check ALI ID : 11
Now check ALI ID : 12
Now check ALI ID : 13
Now check ALI ID : 14
Now check ALI ID : 15
Now check ALI ID : 16
Now check ALI ID : 17
Now check ALI ID : 18
Now check ALI ID : 19
Now check ALI ID : 20
Now check ALI ID : 21
Now check ALI ID : 22
Now check ALI ID : 23
Now check ALI ID : 24
Now check ALI ID : 25
Now check ALI ID : 26
Now check ALI ID : 27
Now check ALI ID : 28
Now check ALI ID : 29
Now check ALI ID : 30
Now check ALI ID : 31
Now check ALI ID : 32
Now check ALI ID : 33
Now check ALI ID : 34
Now check ALI ID : 35
Now check ALI ID : 36
Now check ALI ID : 37
Now check ALI ID : 38
Now check ALI ID : 39
Now check ALI ID : 40
Now check ALI ID : 41
Now check ALI ID : 42
Now check ALI ID : 43
Now ch

Now check ALI ID : 360
Now check ALI ID : 361
Now check ALI ID : 362
Now check ALI ID : 363
Now check ALI ID : 364
Now check ALI ID : 365
Now check ALI ID : 366
Now check ALI ID : 367
Now check ALI ID : 368
Now check ALI ID : 369
Now check ALI ID : 370
Now check ALI ID : 371
Now check ALI ID : 372
Now check ALI ID : 373
Now check ALI ID : 374
Now check ALI ID : 375
Now check ALI ID : 376
Now check ALI ID : 377
Now check ALI ID : 378
Now check ALI ID : 379
Now check ALI ID : 380
Now check ALI ID : 381
Now check ALI ID : 382
Now check ALI ID : 383
Now check ALI ID : 384
Now check ALI ID : 385
Now check ALI ID : 386
Now check ALI ID : 387
Now check ALI ID : 388
Now check ALI ID : 389
Now check ALI ID : 390
Now check ALI ID : 391
Now check ALI ID : 392
Now check ALI ID : 393
Now check ALI ID : 394
Now check ALI ID : 395
Now check ALI ID : 396
Now check ALI ID : 397
Now check ALI ID : 398
Now check ALI ID : 399
Now check ALI ID : 400
Now check ALI ID : 401
Now check ALI ID : 402
Now check A

Now check ALI ID : 736
Now check ALI ID : 737
Now check ALI ID : 738
Now check ALI ID : 739
Now check ALI ID : 740
Now check ALI ID : 741
Now check ALI ID : 742
Now check ALI ID : 743
Now check ALI ID : 744
Now check ALI ID : 745
Now check ALI ID : 746
Now check ALI ID : 747
Now check ALI ID : 748
Now check ALI ID : 749
Now check ALI ID : 750
Now check ALI ID : 751
Now check ALI ID : 752
Now check ALI ID : 753
Now check ALI ID : 754
Now check ALI ID : 755
Now check ALI ID : 756
Now check ALI ID : 757
Now check ALI ID : 758
Now check ALI ID : 759
Now check ALI ID : 760
Now check ALI ID : 761
Now check ALI ID : 762
Now check ALI ID : 763
Now check ALI ID : 764
Now check ALI ID : 765
Now check ALI ID : 766
Now check ALI ID : 767
Now check ALI ID : 768
Now check ALI ID : 769
Now check ALI ID : 770
Now check ALI ID : 771
Now check ALI ID : 772
Now check ALI ID : 773
Now check ALI ID : 774
Now check ALI ID : 775
Now check ALI ID : 776
Now check ALI ID : 777
Now check ALI ID : 778
Now check A

Now check ALI ID : 1092
Now check ALI ID : 1093
Now check ALI ID : 1094
Now check ALI ID : 1095
Now check ALI ID : 1096
Now check ALI ID : 1097
Now check ALI ID : 1098
Now check ALI ID : 1099
Now check ALI ID : 1100
Now check ALI ID : 1101
Now check ALI ID : 1102
Now check ALI ID : 1103
Now check ALI ID : 1104
Now check ALI ID : 1105
Now check ALI ID : 1106
Now check ALI ID : 1107
Now check ALI ID : 1108
Now check ALI ID : 1109
Now check ALI ID : 1110
Now check ALI ID : 1111
Now check ALI ID : 1112
Now check ALI ID : 1113
Now check ALI ID : 1114
Now check ALI ID : 1115
Now check ALI ID : 1116
Now check ALI ID : 1117
Now check ALI ID : 1118
Now check ALI ID : 1119
Now check ALI ID : 1120
Now check ALI ID : 1121
Now check ALI ID : 1122
Now check ALI ID : 1123
Now check ALI ID : 1124
Now check ALI ID : 1125
Now check ALI ID : 1126
Now check ALI ID : 1127
Now check ALI ID : 1128
Now check ALI ID : 1129
Now check ALI ID : 1130
Now check ALI ID : 1131
Now check ALI ID : 1132
Now check ALI ID

Now check ALI ID : 1448
Now check ALI ID : 1449
Now check ALI ID : 1450
Now check ALI ID : 1451
Now check ALI ID : 1452
Now check ALI ID : 1453
Now check ALI ID : 1454
Now check ALI ID : 1455
Now check ALI ID : 1456
Now check ALI ID : 1457
Now check ALI ID : 1458
Now check ALI ID : 1459
Now check ALI ID : 1460
Now check ALI ID : 1461
Now check ALI ID : 1462
Now check ALI ID : 1463
Now check ALI ID : 1464
Now check ALI ID : 1465
Now check ALI ID : 1466
Now check ALI ID : 1467
Now check ALI ID : 1468
Now check ALI ID : 1469
Now check ALI ID : 1470
Now check ALI ID : 1471
Now check ALI ID : 1472
Now check ALI ID : 1473
Now check ALI ID : 1474
Now check ALI ID : 1475
Now check ALI ID : 1476
Now check ALI ID : 1477
Now check ALI ID : 1478
Now check ALI ID : 1479
Now check ALI ID : 1480
Now check ALI ID : 1481
Now check ALI ID : 1482
Now check ALI ID : 1483
Now check ALI ID : 1484
Now check ALI ID : 1485
Now check ALI ID : 1486
Now check ALI ID : 1487
Now check ALI ID : 1488
Now check ALI ID

...

Now check ALI ID : 9395
Now check ALI ID : 9396
Now check ALI ID : 9397
Now check ALI ID : 9398
Now check ALI ID : 9399
Now check ALI ID : 9400
Now check ALI ID : 9401
Now check ALI ID : 9402
Now check ALI ID : 9403
Now check ALI ID : 9404
Now check ALI ID : 9405
Now check ALI ID : 9406
Now check ALI ID : 9407
Now check ALI ID : 9408
Now check ALI ID : 9409
Now check ALI ID : 9410
Now check ALI ID : 9411
Now check ALI ID : 9412
Now check ALI ID : 9413
Now check ALI ID : 9414
Now check ALI ID : 9415
Now check ALI ID : 9416
Now check ALI ID : 9417
Now check ALI ID : 9418
Now check ALI ID : 9419
Now check ALI ID : 9420
Now check ALI ID : 9421
Now check ALI ID : 9422
Now check ALI ID : 9423
Now check ALI ID : 9424
Now check ALI ID : 9425
Now check ALI ID : 9426
Now check ALI ID : 9427
Now check ALI ID : 9428
Now check ALI ID : 9429
Now check ALI ID : 9430
Now check ALI ID : 9431
Now check ALI ID : 9432
Now check ALI ID : 9433
Now check ALI ID : 9434
Now check ALI ID : 9435
Now check ALI ID

Now check ALI ID : 9781
Now check ALI ID : 9782
Now check ALI ID : 9783
Now check ALI ID : 9784
Now check ALI ID : 9785
Now check ALI ID : 9786
Now check ALI ID : 9787
Now check ALI ID : 9788
Now check ALI ID : 9789
Now check ALI ID : 9790
Now check ALI ID : 9791
Now check ALI ID : 9792
Now check ALI ID : 9793
Now check ALI ID : 9794
Now check ALI ID : 9795
Now check ALI ID : 9796
Now check ALI ID : 9797
Now check ALI ID : 9798
Now check ALI ID : 9799
Now check ALI ID : 9800
Now check ALI ID : 9801
Now check ALI ID : 9802
Now check ALI ID : 9803
Now check ALI ID : 9804
Now check ALI ID : 9805
Now check ALI ID : 9806
Now check ALI ID : 9807
Now check ALI ID : 9808
Now check ALI ID : 9809
Now check ALI ID : 9810
Now check ALI ID : 9811
Now check ALI ID : 9812
Now check ALI ID : 9813
Now check ALI ID : 9814
Now check ALI ID : 9815
Now check ALI ID : 9816
Now check ALI ID : 9817
Now check ALI ID : 9818
Now check ALI ID : 9819
Now check ALI ID : 9820
Now check ALI ID : 9821
Now check ALI ID

Now check ALI ID : 10149
Now check ALI ID : 10150
Now check ALI ID : 10151
Now check ALI ID : 10152
Now check ALI ID : 10153
Now check ALI ID : 10154
Now check ALI ID : 10155
Now check ALI ID : 10156
Now check ALI ID : 10157
Now check ALI ID : 10158
Now check ALI ID : 10159
Now check ALI ID : 10160
Now check ALI ID : 10161
Now check ALI ID : 10162
Now check ALI ID : 10163
Now check ALI ID : 10164
Now check ALI ID : 10165
Now check ALI ID : 10166
Now check ALI ID : 10167
Now check ALI ID : 10168
Now check ALI ID : 10169
Now check ALI ID : 10170
Now check ALI ID : 10171
Now check ALI ID : 10172
Now check ALI ID : 10173
Now check ALI ID : 10174
Now check ALI ID : 10175
Now check ALI ID : 10176
Now check ALI ID : 10177
Now check ALI ID : 10178
Now check ALI ID : 10179
Now check ALI ID : 10180
Now check ALI ID : 10181
Now check ALI ID : 10182
Now check ALI ID : 10183
Now check ALI ID : 10184
Now check ALI ID : 10185
Now check ALI ID : 10186
Now check ALI ID : 10187
Now check ALI ID : 10188


Now check ALI ID : 10492
Now check ALI ID : 10493
Now check ALI ID : 10494
Now check ALI ID : 10495
Now check ALI ID : 10496
Now check ALI ID : 10497
Now check ALI ID : 10498
Now check ALI ID : 10499
Now check ALI ID : 10500
Now check ALI ID : 10501
Now check ALI ID : 10502
Now check ALI ID : 10503
Now check ALI ID : 10504
Now check ALI ID : 10505
Now check ALI ID : 10506
Now check ALI ID : 10507
Now check ALI ID : 10508
Now check ALI ID : 10509
Now check ALI ID : 10510
Now check ALI ID : 10511
Now check ALI ID : 10512
Now check ALI ID : 10513
Now check ALI ID : 10514
Now check ALI ID : 10515
Now check ALI ID : 10516
Now check ALI ID : 10517
Now check ALI ID : 10518
Now check ALI ID : 10519
Now check ALI ID : 10520
Now check ALI ID : 10521
Now check ALI ID : 10522
Now check ALI ID : 10523
Now check ALI ID : 10524
Now check ALI ID : 10525
Now check ALI ID : 10526
Now check ALI ID : 10527
Now check ALI ID : 10528
Now check ALI ID : 10529
Now check ALI ID : 10530
Now check ALI ID : 10531


Now check ALI ID : 10822
Now check ALI ID : 10823
Now check ALI ID : 10824
Now check ALI ID : 10825
Now check ALI ID : 10826
Now check ALI ID : 10827
Now check ALI ID : 10828
Now check ALI ID : 10829
Now check ALI ID : 10830
Now check ALI ID : 10831
Now check ALI ID : 10832
Now check ALI ID : 10833
Now check ALI ID : 10834
Now check ALI ID : 10835
Now check ALI ID : 10836
Now check ALI ID : 10837
Now check ALI ID : 10838
Now check ALI ID : 10839
Now check ALI ID : 10840
Now check ALI ID : 10841
Now check ALI ID : 10842
Now check ALI ID : 10843
Now check ALI ID : 10844
Now check ALI ID : 10845
Now check ALI ID : 10846
Now check ALI ID : 10847
Now check ALI ID : 10848
Now check ALI ID : 10849
Now check ALI ID : 10850
Now check ALI ID : 10851
Now check ALI ID : 10852
Now check ALI ID : 10853
Now check ALI ID : 10854
Now check ALI ID : 10855
Now check ALI ID : 10856
Now check ALI ID : 10857
Now check ALI ID : 10858
Now check ALI ID : 10859
Now check ALI ID : 10860
Now check ALI ID : 10861
